### 

In [1]:
pip install feedparser

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

import threading
import queue
import feedparser
import uuid

threads=[]
queues=[queue.Queue(),queue.Queue()]

def extractWords():
    url=='https://timesofindia.indiatimes.com/rssfeeds/1081479906.cms'
    feed=feedparesr.parse(url)
    for entry in feed['entries'][:5]:
        text = entry['title']
        if 'ex' in text:
            continue
        words=nltk.word_tokenize(text)
        data={'uuid':uuid.uuid4(),'input':words}
        queues[0].put(data,True)
        print(">> {}: {}".format(data['uuid'],text))

## 핵심 키워드 추출 ( Keyword Extraction)

## 1. TF-IDF

### 1) 샘플 텍스트

In [2]:
d1="The cat sat on my face. I hate a cat."
d2="The dog sat on my bed. I love a dog."

### 2) sklearn을 활용 TF-IDF

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

document_ls = [d1,d2]

vectorizer=TfidfVectorizer()
tfidf = vectorizer.fit_transform(document_ls)

word2id = defaultdict(lambda: 0)
for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx

### 3) data frame으로 변환하여 출력

In [10]:
import pandas as pd
count_vect_df = pd.DataFrame(tfidf.todense(),columns=vectorizer.get_feature_names())
count_vect_df

,bed,cat,dog,face,hate,love,my,on,sat,the
0,0.000000,0.706006,0.000000,0.353003,0.353003,0.000000,0.251164,0.251164,0.251164,0.251164
1,0.353003,0.000000,0.706006,0.000000,0.000000,0.353003,0.251164,0.251164,0.251164,0.251164


### 4) TF-IDF score가 높은 순으로 출력...!!!

In [11]:
import numpy as np

feature_array=np.array(vectorizer.get_feature_names())
tfidf_sorting = np.argsort(tfidf[0].toarray()).flatten()[::-1] # 역순으로 출력

n=3
top_n=feature_array[tfidf_sorting][:n]

top_n

array(['cat', 'hate', 'face'], dtype='<U4')

## 2. Textrank

In [ ]:
#
50번만큼 돌림.  50번만큼만 계산...!!!
threshold =     만큼 차이나면 이제 그만 돌아라...!!!

In [15]:
#https://www.businessinsider.sg/faang-stocks-are-a-dead-trade-for-the-next-several-months-2018-3/amp/

Text = "The FAANG stocks won’t see much more growth in the near future, according to Bill Studebaker, founder and Chief Investment Officer of Robo Global. \
Studebaker argues we are seeing a 'reallocation' that will continue from large-cap tech stocks into market-weight stocks. \
The FAANG stocks have had a rough few weeks, and have been hit hard since March 12. \
One FAANG to look out for, in the midst of all this, is Amazon, according to Studebaker. \
The stock market is seeing a 'reallocation' out of FAANG stocks, which are not where the smart money is, founder and Chief Investment Officer of Robo Global Bill Studebaker told Business Insider. \
The FAANG stocks (Facebook, Apple, Amazon, Netflix, Google) are all down considerably since March 12, a trend that accelerated when news of a massive Facebook data scandal broke, sending the tech-heavy Nasdaq into a downward frenzy. \
Investors are wondering what’s next. \
And what’s next isn’t good news for FAANG stock optimists, Studebaker thinks. 'This is a dead trade' for the next several months, he said. 'I wouldn’t expect there to be a lot of performance attribution coming from the FAANG stocks,' he added. That is, if the stock market is to see gains in the next several months, they will largely not come from the big tech companies. \
The market is seeing a 'reallocation out of large-cap technology, into other parts of the market,' he said. And this trend could continue for the foreseeable future. 'When you get these reallocation trades, a de-risking, this can go on for months and months.' The FAANG’s are pricey stocks, he said, pointing out that investors will 'factor in the law of big numbers,' he said. 'Just because they’re big cap doesn’t mean they’re safe,' he added. \
Still, he doesn’t necessarily think that investors are going to shift drastically into value stocks. 'With an increasingly favorable macro backdrop, you have strong growth demand.' \
Studebaker, who runs an artificial intelligence and robotics exchange-traded fund with $4 billion in assets under management, thinks that AI and robotics are better areas of growth. His ETF is up 27% in the past year, while the FAANG stocks are also largely up over that same span, even if they are down since March 12. \
While many point to artificial intelligence as an area that will be a boost to Google and Amazon, Studebaker doesn’t see that as a sign of significant growth for the FAANGs. He pointed out that 'eighty to ninety percent of their businesses are still search,' and that 'AI doesn’t really move the needle on the business.' He also said 'the revenue mix [attributable to AI] in those businesses are insignificant.' \
And while he’s not bullish on FAANG’s, he does say that the one FAANG to still watch out for is Amazon, simply because ecommerce still represents a small portion of the global retail market, giving the company room to grow." 



### 1) 토큰화 (Tokenization)

In [16]:
import nltk
import string
from nltk.tokenize import TreebankWordTokenizer

nltk.download('punkt')  # 토큰 분리할 때 쓰는 것...!!! 기준이 됨..!!

text=TreebankWordTokenizer().tokenize(Text)

print("Tokenized Text: |n")
print(text)

Tokenized Text: |n
['The', 'FAANG', 'stocks', 'won', '’', 't', 'see', 'much', 'more', 'growth', 'in', 'the', 'near', 'future', ',', 'according', 'to', 'Bill', 'Studebaker', ',', 'founder', 'and', 'Chief', 'Investment', 'Officer', 'of', 'Robo', 'Global.', 'Studebaker', 'argues', 'we', 'are', 'seeing', 'a', "'reallocation", "'", 'that', 'will', 'continue', 'from', 'large-cap', 'tech', 'stocks', 'into', 'market-weight', 'stocks.', 'The', 'FAANG', 'stocks', 'have', 'had', 'a', 'rough', 'few', 'weeks', ',', 'and', 'have', 'been', 'hit', 'hard', 'since', 'March', '12.', 'One', 'FAANG', 'to', 'look', 'out', 'for', ',', 'in', 'the', 'midst', 'of', 'all', 'this', ',', 'is', 'Amazon', ',', 'according', 'to', 'Studebaker.', 'The', 'stock', 'market', 'is', 'seeing', 'a', "'reallocation", "'", 'out', 'of', 'FAANG', 'stocks', ',', 'which', 'are', 'not', 'where', 'the', 'smart', 'money', 'is', ',', 'founder', 'and', 'Chief', 'Investment', 'Officer', 'of', 'Robo', 'Global', 'Bill', 'Studebaker', 'told

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jungsk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 2) 품사부착 (Pos Tagging)
* 토큰화된 텍스트에 품사 부착

In [18]:
nltk.download('averaged_perceptron_tagger')
POS_tag=nltk.pos_tag(text) #품사 부착

print(POS_tag)

[('The', 'DT'), ('FAANG', 'NNP'), ('stocks', 'NNS'), ('won', 'VBD'), ('’', 'JJ'), ('t', 'NN'), ('see', 'VBP'), ('much', 'RB'), ('more', 'JJR'), ('growth', 'NN'), ('in', 'IN'), ('the', 'DT'), ('near', 'JJ'), ('future', 'NN'), (',', ','), ('according', 'VBG'), ('to', 'TO'), ('Bill', 'NNP'), ('Studebaker', 'NNP'), (',', ','), ('founder', 'NN'), ('and', 'CC'), ('Chief', 'NNP'), ('Investment', 'NNP'), ('Officer', 'NNP'), ('of', 'IN'), ('Robo', 'NNP'), ('Global.', 'NNP'), ('Studebaker', 'NNP'), ('argues', 'VBZ'), ('we', 'PRP'), ('are', 'VBP'), ('seeing', 'VBG'), ('a', 'DT'), ("'reallocation", 'NN'), ("'", 'POS'), ('that', 'WDT'), ('will', 'MD'), ('continue', 'VB'), ('from', 'IN'), ('large-cap', 'JJ'), ('tech', 'NN'), ('stocks', 'NNS'), ('into', 'IN'), ('market-weight', 'JJ'), ('stocks.', 'NN'), ('The', 'DT'), ('FAANG', 'NNP'), ('stocks', 'NNS'), ('have', 'VBP'), ('had', 'VBD'), ('a', 'DT'), ('rough', 'JJ'), ('few', 'JJ'), ('weeks', 'NNS'), (',', ','), ('and', 'CC'), ('have', 'VBP'), ('been',

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Jungsk\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


### 3) 표제어 추출 (Lemmatization)

In [19]:
from nltk.corpus import wordnet
nltk.download('wordnet')

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jungsk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

lemmatized_text=[] # 각 토큰의 표제어 추출
for word in POS_tag:
    lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0],pos=get_wordnet_pos(word[1]))))
    
print(lemmatized_text)    

['The', 'FAANG', 'stock', 'win', '’', 't', 'see', 'much', 'more', 'growth', 'in', 'the', 'near', 'future', ',', 'accord', 'to', 'Bill', 'Studebaker', ',', 'founder', 'and', 'Chief', 'Investment', 'Officer', 'of', 'Robo', 'Global.', 'Studebaker', 'argue', 'we', 'be', 'see', 'a', "'reallocation", "'", 'that', 'will', 'continue', 'from', 'large-cap', 'tech', 'stock', 'into', 'market-weight', 'stocks.', 'The', 'FAANG', 'stock', 'have', 'have', 'a', 'rough', 'few', 'week', ',', 'and', 'have', 'be', 'hit', 'hard', 'since', 'March', '12.', 'One', 'FAANG', 'to', 'look', 'out', 'for', ',', 'in', 'the', 'midst', 'of', 'all', 'this', ',', 'be', 'Amazon', ',', 'accord', 'to', 'Studebaker.', 'The', 'stock', 'market', 'be', 'see', 'a', "'reallocation", "'", 'out', 'of', 'FAANG', 'stock', ',', 'which', 'be', 'not', 'where', 'the', 'smart', 'money', 'be', ',', 'founder', 'and', 'Chief', 'Investment', 'Officer', 'of', 'Robo', 'Global', 'Bill', 'Studebaker', 'tell', 'Business', 'Insider.', 'The', 'FAANG

### 4)불용어(stopwords) 처리 및 불필요한 품사 제거

In [21]:
stopwords= [] # 불용어 배열

#추출 키워드 대상이 되는 품사 지정 (N: 명사 J : 형용사)
wanted_POS = ['NN','NNS','NNP','NNPS','JJ','JJR','JJS']

#추출 키워드 대상 품사가 아닌 토큰은 불용어로 등록

for word in POS_tag:
    if word[1]not in wanted_POS:
        stopwords.append(word[0])
        
#punctuation 을 불용어로 추가
punctuations = list(str(string.punctuation))
stopwords=stopwords + punctuations

#사용자 정의 토큰을 불용어로 추가
stopwords_plus = ['t','isn']
stopwords=stopwords+stopwords_plus
stopwords=set(stopwords)

processed_text=[]
for word in lemmatized_text:
    if word not in stopwords:
        processed_text.append(word)
        
print(processed_text)

['FAANG', 'stock', 'win', 'more', 'growth', 'near', 'future', 'accord', 'Bill', 'Studebaker', 'founder', 'Chief', 'Investment', 'Officer', 'Robo', 'Global.', 'Studebaker', 'argue', "'reallocation", 'large-cap', 'tech', 'stock', 'market-weight', 'stocks.', 'FAANG', 'stock', 'rough', 'few', 'week', 'hard', 'March', 'One', 'FAANG', 'midst', 'Amazon', 'accord', 'Studebaker.', 'stock', 'market', "'reallocation", 'FAANG', 'stock', 'smart', 'money', 'founder', 'Chief', 'Investment', 'Officer', 'Robo', 'Global', 'Bill', 'Studebaker', 'tell', 'Business', 'Insider.', 'FAANG', 'stock', 'Facebook', 'Apple', 'Amazon', 'Netflix', 'Google', 'March', 'trend', 'accelerate', 'news', 'massive', 'Facebook', 'data', 'scandal', 'break', 'send', 'tech-heavy', 'Nasdaq', 'downward', 'frenzy.', 'Investors', 'wonder', 'next', 'good', 'news', 'FAANG', 'stock', 'optimist', 'Studebaker', 'thinks.', "'This", 'dead', 'trade', 'next', 'several', 'month', 'lot', 'performance', 'attribution', 'FAANG', 'stock', 'stock', 

In [22]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

###  5) Unique한 토큰 목록 생성

* 그래프 생성을 위해서, Unique한 토큰 목록 생성

In [23]:
vocabulary=list(str(processed_text))
print(vocabulary)

['[', "'", 'F', 'A', 'A', 'N', 'G', "'", ',', ' ', "'", 's', 't', 'o', 'c', 'k', "'", ',', ' ', "'", 'w', 'i', 'n', "'", ',', ' ', "'", 'm', 'o', 'r', 'e', "'", ',', ' ', "'", 'g', 'r', 'o', 'w', 't', 'h', "'", ',', ' ', "'", 'n', 'e', 'a', 'r', "'", ',', ' ', "'", 'f', 'u', 't', 'u', 'r', 'e', "'", ',', ' ', "'", 'a', 'c', 'c', 'o', 'r', 'd', "'", ',', ' ', "'", 'B', 'i', 'l', 'l', "'", ',', ' ', "'", 'S', 't', 'u', 'd', 'e', 'b', 'a', 'k', 'e', 'r', "'", ',', ' ', "'", 'f', 'o', 'u', 'n', 'd', 'e', 'r', "'", ',', ' ', "'", 'C', 'h', 'i', 'e', 'f', "'", ',', ' ', "'", 'I', 'n', 'v', 'e', 's', 't', 'm', 'e', 'n', 't', "'", ',', ' ', "'", 'O', 'f', 'f', 'i', 'c', 'e', 'r', "'", ',', ' ', "'", 'R', 'o', 'b', 'o', "'", ',', ' ', "'", 'G', 'l', 'o', 'b', 'a', 'l', '.', "'", ',', ' ', "'", 'S', 't', 'u', 'd', 'e', 'b', 'a', 'k', 'e', 'r', "'", ',', ' ', "'", 'a', 'r', 'g', 'u', 'e', "'", ',', ' ', '"', "'", 'r', 'e', 'a', 'l', 'l', 'o', 'c', 'a', 't', 'i', 'o', 'n', '"', ',', ' ', "'", 'l',

### 6) 그래프 생성 (weighted edge 계산)

* TextRank는 그래프 기반 모델
* 각 단어(토큰)은 그래프의 노드(vertex)
* weighted_edge 행렬은 노드가 가중치 정보를 담고 있음
* weighted_edge [i][j]는 i번째 단어와 j번째 단어의 가중치를 의미
* weighted_edgd [i][j]가 0인 이유는 노드간 연결이 없음을 의미
* 모든 노드는 1로 초기화

In [34]:
import numpy as np
import math
vocab_len = len(vocabulary)

# 토큰별로 그래프 edge를 Matrix 형태로 생성
weighted_edge = np.zeros((vocab_len,vocab_len),dtype=np.float32)

#각 토큰 노트별로 점수계산을 위한 배열 생성
score = np.zeros((vocab_len),dtype=np.float32)

#coocurrence를 판단하기 위한 window 사이즈 설정
window_size=3
covered_coocurrences=[]

for i in range(0,vocab_len):
    score[i]=1
    for j  in range(0,vocab_len):
        if j==i:
            continue
        else :
            for window_start in range(0,(len(processed_text)-window_size)):
                
                window_end = window_start+window_size
                
                window=processed_text[window_start:window_end]
                
                # 탐색하고 있는 두 단어가 윈도우에 동시 등장할 경우
                if(vocabulary[i] in window) and (vocabulary[j] in window):
                    
                    index_of_i = window_start + window.index(vocabulary[i])
                    index_of_j = window_start + window.index(vocabulary[j])
                    
                    if [index_of_i,index_of_j] not in covered_cooncurrences:
                        weighted_edge[i][j]+=1/math.fabs(index_of_i-index_of_j)
                        covered_coocurrences.append([index_of_i,index_of_j])

#윈도우 사이즈만큼만 


#마지막 토큰에서, 윈도우사이즈만큼만 탐색하도록...!?!

KeyboardInterrupt: 

### 7) 각 노드의 score 계산

* 각 노드와 연결된 weighted edge의 값을 합산

In [35]:
inout=np.zeros((vocab_len),dtype=np.float32)

for i in range(0,vocab_len):
    for j in range(0,vocab_len):
        inout[i] +=weighted_edge[i][j]

In [37]:
MAX_ITERATIONS = 50
d=0.85
threashod=0.0001 #convergence threshold

for iter in range(0,MAX_ITERATIONS):
    prev_score = np.copy(score)
    
    for i in range(0,vocab_len):
        
        summation=0
        for j in range(0,vocab_len):
            if weighted_edge[i][j] !=0:
                summation += (weighted_edge[i][j]/inout[j])*score[j]
                
        score[i]=(1-d)+d*(summation)
        
    if np.sum(np.fabs(prev_score-score)) <=threashod: #convergence condition
        break

In [38]:
for i in range(0,10): #vocab_len
    print("Score of "+vocabulary[i]+": "+str(score[i]))

Score of [: 0.15
Score of ': 0.15
Score of F: 0.15
Score of A: 0.15
Score of A: 0.15
Score of N: 0.15
Score of G: 0.15
Score of ': 0.15
Score of ,: 0.15
Score of  : 0.15


### 8) 핵심 단어 추출

In [39]:
sorted_index = np.flip(np.argsort(score),0)

keywords_num=10

print("Keywords:|n")

for i in range(0,keywords_num):
    print(str(vocabulary[sorted_index[i]])+" : "+str(score[sorted_index[i]]))

Keywords:|n
] : 0.15
F : 0.15
c : 0.15
e : 0.15
b : 0.15
o : 0.15
o : 0.15
k : 0.15
' : 0.15
, : 0.15


In [41]:
sorted_index
np.argsort(score)

array([   0, 1400, 1399, ...,  688,  686, 2089], dtype=int64)

In [42]:
np.flip?

## 2.2 TextRank 핵심 구 추출

1) 불용어를 기준으로 구 추출  - stop words 를 기준으로..!

In [43]:
phrases=[]

phrase = " "

for word in lemmatized_text:
    if word in stopwords:
        if phrases !=" ":
            phrases.append(str(phrase).strip().split())
        phrase=" "
    elif word not in stopwords:
        phrase+=str(word)
        phrase+=" "
        
print(phrases)

[[], ['FAANG', 'stock', 'win'], [], [], [], ['more', 'growth'], [], ['near', 'future'], ['accord'], ['Bill', 'Studebaker'], ['founder'], ['Chief', 'Investment', 'Officer'], ['Robo', 'Global.', 'Studebaker', 'argue'], [], [], [], ["'reallocation"], [], [], [], [], ['large-cap', 'tech', 'stock'], ['market-weight', 'stocks.'], ['FAANG', 'stock'], [], [], ['rough', 'few', 'week'], [], [], [], [], ['hard'], ['March'], ['One', 'FAANG'], [], [], [], [], [], [], ['midst'], [], [], [], [], ['Amazon'], ['accord'], ['Studebaker.'], ['stock', 'market'], [], [], ["'reallocation"], [], [], ['FAANG', 'stock'], [], [], [], [], [], ['smart', 'money'], [], ['founder'], ['Chief', 'Investment', 'Officer'], ['Robo', 'Global', 'Bill', 'Studebaker', 'tell', 'Business', 'Insider.'], ['FAANG', 'stock'], ['Facebook'], ['Apple'], ['Amazon'], ['Netflix'], ['Google'], [], [], [], [], [], ['March'], [], [], ['trend'], ['accelerate'], ['news'], [], ['massive', 'Facebook', 'data', 'scandal', 'break'], ['send'], ['tec

In [44]:
unique_phrases = []

for phrase in phrases:
    if phrase not in unique_phrases:
        unique_phrases.append(phrase)
        
print(unique_phrases)

[[], ['FAANG', 'stock', 'win'], ['more', 'growth'], ['near', 'future'], ['accord'], ['Bill', 'Studebaker'], ['founder'], ['Chief', 'Investment', 'Officer'], ['Robo', 'Global.', 'Studebaker', 'argue'], ["'reallocation"], ['large-cap', 'tech', 'stock'], ['market-weight', 'stocks.'], ['FAANG', 'stock'], ['rough', 'few', 'week'], ['hard'], ['March'], ['One', 'FAANG'], ['midst'], ['Amazon'], ['Studebaker.'], ['stock', 'market'], ['smart', 'money'], ['Robo', 'Global', 'Bill', 'Studebaker', 'tell', 'Business', 'Insider.'], ['Facebook'], ['Apple'], ['Netflix'], ['Google'], ['trend'], ['accelerate'], ['news'], ['massive', 'Facebook', 'data', 'scandal', 'break'], ['send'], ['tech-heavy', 'Nasdaq'], ['downward', 'frenzy.', 'Investors'], ['wonder'], ['next'], ['good', 'news'], ['FAANG', 'stock', 'optimist'], ['Studebaker', 'thinks.', "'This"], ['dead', 'trade'], ['next', 'several', 'month'], ['lot'], ['performance', 'attribution'], ['gain'], ['big', 'tech'], ['market'], ['large-cap', 'technology']

In [45]:
for word in vocabulary:
    #print word
    for phrase in unique_phrases:
        if (word in phrase) and ([word] in unique_phrases) and len((phrase)>1):
            unique_phrases.remove([word])
            
    print(unique_phrases)

[[], ['FAANG', 'stock', 'win'], ['more', 'growth'], ['near', 'future'], ['accord'], ['Bill', 'Studebaker'], ['founder'], ['Chief', 'Investment', 'Officer'], ['Robo', 'Global.', 'Studebaker', 'argue'], ["'reallocation"], ['large-cap', 'tech', 'stock'], ['market-weight', 'stocks.'], ['FAANG', 'stock'], ['rough', 'few', 'week'], ['hard'], ['March'], ['One', 'FAANG'], ['midst'], ['Amazon'], ['Studebaker.'], ['stock', 'market'], ['smart', 'money'], ['Robo', 'Global', 'Bill', 'Studebaker', 'tell', 'Business', 'Insider.'], ['Facebook'], ['Apple'], ['Netflix'], ['Google'], ['trend'], ['accelerate'], ['news'], ['massive', 'Facebook', 'data', 'scandal', 'break'], ['send'], ['tech-heavy', 'Nasdaq'], ['downward', 'frenzy.', 'Investors'], ['wonder'], ['next'], ['good', 'news'], ['FAANG', 'stock', 'optimist'], ['Studebaker', 'thinks.', "'This"], ['dead', 'trade'], ['next', 'several', 'month'], ['lot'], ['performance', 'attribution'], ['gain'], ['big', 'tech'], ['market'], ['large-cap', 'technology']

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### 2) 각 구의 Score 계산

* 앞서 산출한 단어별 점수를 합산


In [46]:
phrase_scores= []
keywords = []

for phrase in unique_phrases:
    phrase_score=0
    keyword=''
    for word in phrase:
        keyword += str(word)
        keyword += " "
        phrase_score +=score[vocabulary.index(word)]
        
    phrase_scores.append(phrase_score)
    keywords.append(keyword.strip())

i=0
for keyword in keywords:
    print("Keyword: '" +str(keyword)+ "', Score: " +str(phrase_scores[i]))
    
    i+=1

ValueError: 'FAANG' is not in list

### 3) 각 구를 Score로 정렬하여 핵심 구 추출

In [47]:
sorted_index = np.flip(np.argsort(phrase_scores),0)

keywords_sum= 10

print("Keywords: |n")

for i in range(0,keywords_sum):
    print(str(keywords[sorted_index[i]])+",")

Keywords: |n
,


IndexError: index 1 is out of bounds for axis 0 with size 1

### 2,3 gensim Textrank

In [51]:
from gensim.summarization import keywords

keywords(Text).split('\n')

['stocks',
 'stock',
 'studebaker',
 'trade',
 'trades',
 'amazon',
 'tech',
 'attribution',
 'attributable',
 'cap',
 'facebook',
 'market',
 'future',
 'growth',
 'thinks',
 'think',
 'investment',
 'frenzy',
 'big',
 'global',
 'favorable macro']

In [52]:
from gensim.summarization.summarizer import summarize

print(summarize(Text))

The FAANG stocks won’t see much more growth in the near future, according to Bill Studebaker, founder and Chief Investment Officer of Robo Global.
Studebaker argues we are seeing a 'reallocation' that will continue from large-cap tech stocks into market-weight stocks.
The stock market is seeing a 'reallocation' out of FAANG stocks, which are not where the smart money is, founder and Chief Investment Officer of Robo Global Bill Studebaker told Business Insider.
